In [52]:
import requests
import pandas as pd
import numpy as np
import random
import folium
import json
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [76]:
## Foursquare developer account

CLIENT_ID = 'id'
CLIENT_SECRET = 'secret'
VERSION = '20180604'
LIMIT = 30
print('My_Foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My_Foursquare credentails:
CLIENT_ID: id
CLIENT_SECRET:secret


In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [5]:
with open("'newyork_data.json'") as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

Empty DataFrame
Columns: [Borough, Neighborhood, Latitude, Longitude]
Index: []

In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [14]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [15]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [17]:
neighborhoods.loc[0, 'Neighborhood']

'Wakefield'

In [18]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=AJ3VPP1DWBF0J12LZGRXTTYPNXHAUS2L4PY5LCP4HQCAPDDN&client_secret=GSD53HFGA5LPASWKWMMJJZLRSUXJ3QA4PMXL51VEGNI4ARP5&v=20180604&ll=40.89470517661,-73.84720052054902&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e077e3907e700210866f7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wakefield',
  'headerFullLocation': 'Wakefield, Bronx',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 40.899205181110005,
    'lng': -73.84125857127495},
   'sw': {'lat': 40.89020517211, 'lng': -73.8531424698231}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c537892fd2ea593cb077a28',
       'name': 'Lollipops Gelato',
       'location': {'address': '4120 Baychester Ave',
        'crossStreet': 'Edenwald & Bussing Ave',
        'lat': 40.894123150205274,
        'lng': -73.84589162362325,
        'labeledLatLn

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name      categories        lat        lng
0  Lollipops Gelato    Dessert Shop  40.894123 -73.845892
1          Rite Aid        Pharmacy  40.896649 -73.844846
2  Carvel Ice Cream  Ice Cream Shop  40.890487 -73.848568
3           Dunkin'      Donut Shop  40.890459 -73.849089
4             Shell     Gas Station  40.894187 -73.845862

In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
neighborhood_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [30]:
print(neighborhood_venues.shape)
neighborhood_venues.head()

(10245, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Wakefield              40.894705              -73.847201   
1    Wakefield              40.894705              -73.847201   
2    Wakefield              40.894705              -73.847201   
3    Wakefield              40.894705              -73.847201   
4    Wakefield              40.894705              -73.847201   

              Venue  Venue Latitude  Venue Longitude  Venue Category  
0  Lollipops Gelato       40.894123       -73.845892    Dessert Shop  
1          Rite Aid       40.896649       -73.844846        Pharmacy  
2  Carvel Ice Cream       40.890487       -73.848568  Ice Cream Shop  
3           Dunkin'       40.890459       -73.849089      Donut Shop  
4             Shell       40.894187       -73.845862     Gas Station

In [31]:
neighborhood_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                               
Allerton                               33                      33     33   
Annadale                               11                      11     11   
Arden Heights                           6                       6      6   
Arlington                               5                       5      5   
Arrochar                               24                      24     24   
Arverne                                18                      18     18   
Astoria                               100                     100    100   
Astoria Heights                        13                      13     13   
Auburndale                             19                      19     19   
Bath Beach                             53                      53     53   
Battery Park City                      95                      95     95   
Bay Ridge                              86                      86     86   
Bay Terrace                            47                      47     47   
Baychester                             23                      23     23   
Bayside                                74                      74     74   
Bayswater                               3                       3      3   
Bedford Park                           37                      37     37   
Bedford Stuyvesant                     26                      26     26   
Beechhurst                             15                      15     15   
Bellaire                               10                      10     10   
Belle Harbor                           16                      16     16   
Bellerose                              20                      20     20   
Belmont                               100                     100    100   
Bensonhurst                            29                      29     29   
Bergen Beach                            8                       8      8   
Blissville                             19                      19     19   
Bloomfield                              5                       5      5   
Boerum Hill                            86                      86     86   
Borough Park                           18                      18     18   
Breezy Point                            3                       3      3   
...                                   ...                     ...    ...   
Travis                                 14                      14     14   
Tribeca                               100                     100    100   
Tudor City                             79                      79     79   
Turtle Bay                            100                     100    100   
Unionport                              23                      23     23   
University Heights                     23                      23     23   
Upper East Side                       100                     100    100   
Upper West Side                       100                     100    100   
Utopia                                 15                      15     15   
Van Nest                               17                      17     17   
Vinegar Hill                           28                      28     28   
Wakefield                               8                       8      8   
Washington Heights                     83                      83     83   
Weeksville                             18                      18     18   
West Brighton                          39                      39     39   
West Farms                             24                      24     24   
West Village                          100                     100    100   
Westchester Square                     32                      32     32   
Westerleigh                             3                       3      3   
Whitestone                              4                       4   

In [33]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 433 uniques categories.


In [35]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ["Neighborhood"] + [x for x in neighborhood_onehot.columns if x!="Neighborhood"]
# [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

Neighborhood  Accessories Store  Adult Boutique  Afghan Restaurant  \
0    Wakefield                  0               0                  0   
1    Wakefield                  0               0                  0   
2    Wakefield                  0               0                  0   
3    Wakefield                  0               0                  0   
4    Wakefield                  0               0                  0   

   African Restaurant  American Restaurant  Antique Shop  Arcade  \
0                   0                    0             0       0   
1                   0                    0             0       0   
2                   0                    0             0       0   
3                   0                    0             0       0   
4                   0                    0             0       0   

   Arepa Restaurant  Argentinian Restaurant  ...  Warehouse Store  \
0                 0                       0  ...                0   
1                 0                       0  ...                0   
2                 0                       0  ...                0   
3                 0                       0  ...                0   
4                 0                       0  ...                0   

   Waste Facility  Waterfront  Weight Loss Center  Whisky Bar  Wine Bar  \
0               0           0                   0           0         0   
1               0           0                   0           0         0   
2               0           0                   0           0         0   
3               0           0                   0           0         0   
4               0           0                   0           0         0   

   Wine Shop  Wings Joint  Women's Store  Yoga Studio  
0          0            0              0            0  
1          0            0              0            0  
2          0            0              0            0  
3          0            0              0            0  
4          0            0              0            0  

[5 rows x 433 columns]

In [36]:
neighborhood_onehot.shape

(10245, 433)

In [39]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped.head()

Neighborhood  Accessories Store  Adult Boutique  Afghan Restaurant  \
0       Allerton                0.0             0.0                0.0   
1       Annadale                0.0             0.0                0.0   
2  Arden Heights                0.0             0.0                0.0   
3      Arlington                0.0             0.0                0.0   
4       Arrochar                0.0             0.0                0.0   

   African Restaurant  American Restaurant  Antique Shop  Arcade  \
0                 0.0             0.000000           0.0     0.0   
1                 0.0             0.090909           0.0     0.0   
2                 0.0             0.000000           0.0     0.0   
3                 0.0             0.200000           0.0     0.0   
4                 0.0             0.000000           0.0     0.0   

   Arepa Restaurant  Argentinian Restaurant  ...  Warehouse Store  \
0               0.0                     0.0  ...              0.0   
1               0.0                     0.0  ...              0.0   
2               0.0                     0.0  ...              0.0   
3               0.0                     0.0  ...              0.0   
4               0.0                     0.0  ...              0.0   

   Waste Facility  Waterfront  Weight Loss Center  Whisky Bar  Wine Bar  \
0             0.0         0.0                 0.0         0.0       0.0   
1             0.0         0.0                 0.0         0.0       0.0   
2             0.0         0.0                 0.0         0.0       0.0   
3             0.0         0.0                 0.0         0.0       0.0   
4             0.0         0.0                 0.0         0.0       0.0   

   Wine Shop  Wings Joint  Women's Store  Yoga Studio  
0        0.0          0.0            0.0          0.0  
1        0.0          0.0            0.0          0.0  
2        0.0          0.0            0.0          0.0  
3        0.0          0.0            0.0          0.0  
4        0.0          0.0            0.0          0.0  

[5 rows x 433 columns]

In [38]:
neighborhood_grouped.shape

(301, 433)

In [41]:
num_top_venues = 5

for popular in neighborhood_grouped['Neighborhood']:
    print("----"+popular+"----")
    temp = neighborhood_grouped[neighborhood_grouped['Neighborhood'] == popular].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print()

----Allerton----
                venue  freq
0         Pizza Place  0.15
1  Chinese Restaurant  0.09
2       Deli / Bodega  0.06
3         Supermarket  0.06
4      Cosmetics Shop  0.06

----Annadale----
              venue  freq
0       Pizza Place  0.18
1      Dance Studio  0.09
2  Sushi Restaurant  0.09
3     Train Station  0.09
4        Sports Bar  0.09

----Arden Heights----
           venue  freq
0       Pharmacy  0.17
1  Deli / Bodega  0.17
2       Bus Stop  0.17
3   Home Service  0.17
4    Coffee Shop  0.17

----Arlington----
                 venue  freq
0             Bus Stop   0.4
1        Deli / Bodega   0.2
2  American Restaurant   0.2
3         Intersection   0.2
4    Accessories Store   0.0

----Arrochar----
                venue  freq
0            Bus Stop  0.12
1  Italian Restaurant  0.08
2         Pizza Place  0.08
3          Bagel Shop  0.08
4        Liquor Store  0.08

----Arverne----
            venue  freq
0       Surf Spot  0.22
1   Metro Station  0.11
2  Sandwich 

              venue  freq
0     Big Box Store  0.06
1    Cosmetics Shop  0.06
2       Supermarket  0.03
3  Video Game Store  0.03
4    Clothing Store  0.03

----Chelsea----
                venue  freq
0         Coffee Shop  0.06
1              Bakery  0.05
2             Theater  0.04
3      Ice Cream Shop  0.04
4  Italian Restaurant  0.04

----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.10
1         Cocktail Bar  0.04
2  American Restaurant  0.04
3   Salon / Barbershop  0.03
4    Hotpot Restaurant  0.03

----City Island----
                    venue  freq
0         Harbor / Marina  0.11
1           Deli / Bodega  0.07
2      Seafood Restaurant  0.07
3  Thrift / Vintage Store  0.07
4           Boat or Ferry  0.07

----City Line----
                 venue  freq
0           Donut Shop  0.08
1             Pharmacy  0.05
2  Fried Chicken Joint  0.05
3       Clothing Store  0.05
4           Shoe Store  0.05

----Civic Center----
                  venue  freq
0  Gym /

               venue  freq
0               Food   1.0
1  Accessories Store   0.0
2  Paella Restaurant   0.0
3           Pharmacy   0.0
4          Pet Store   0.0

----Erasmus----
                  venue  freq
0  Caribbean Restaurant  0.15
1           Yoga Studio  0.05
2           Music Venue  0.05
3     Mobile Phone Shop  0.05
4     Health Food Store  0.05

----Far Rockaway----
                venue  freq
0       Grocery Store  0.09
1  Chinese Restaurant  0.09
2         Pizza Place  0.09
3         Bus Station  0.06
4         Supermarket  0.06

----Fieldston----
                  venue  freq
0                 River  0.33
1           Bus Station  0.33
2                 Plaza  0.33
3     Accessories Store  0.00
4  Pakistani Restaurant  0.00

----Financial District----
                 venue  freq
0          Coffee Shop  0.10
1  American Restaurant  0.05
2                Hotel  0.04
3          Pizza Place  0.04
4                  Gym  0.04

----Flatbush----
                  venue  freq
0 

           venue  freq
0     Restaurant  0.08
1      BBQ Joint  0.08
2   Home Service  0.08
3  Grocery Store  0.08
4   Gourmet Shop  0.08

----Inwood----
                venue  freq
0                Café  0.07
1  Mexican Restaurant  0.07
2          Restaurant  0.05
3              Lounge  0.05
4              Bakery  0.04

----Jackson Heights----
                       venue  freq
0  Latin American Restaurant  0.11
1        Peruvian Restaurant  0.07
2  South American Restaurant  0.06
3          Mobile Phone Shop  0.05
4                     Bakery  0.05

----Jamaica Center----
                   venue  freq
0      Mobile Phone Shop  0.09
1         Clothing Store  0.07
2  Performing Arts Venue  0.04
3     Mexican Restaurant  0.04
4       Department Store  0.04

----Jamaica Estates----
               venue  freq
0        Bus Station  0.25
1       Antique Shop  0.25
2       Intersection  0.25
3  Indian Restaurant  0.25
4  Accessories Store  0.00

----Jamaica Hills----
                  venue

            venue  freq
0   Grocery Store  0.17
1  Clothing Store  0.08
2  Sandwich Place  0.08
3   Deli / Bodega  0.08
4     Supermarket  0.08

----Murray Hill----
                 venue  freq
0    Korean Restaurant  0.16
1          Coffee Shop  0.05
2                  Bar  0.03
3  American Restaurant  0.03
4                Hotel  0.03

----Neponsit----
                  venue  freq
0                 Beach   1.0
1     Accessories Store   0.0
2  Pakistani Restaurant   0.0
3              Pharmacy   0.0
4             Pet Store   0.0

----New Brighton----
            venue  freq
0        Bus Stop  0.33
1   Deli / Bodega  0.25
2            Park  0.17
3  Discount Store  0.08
4   Bowling Alley  0.08

----New Dorp----
                venue  freq
0  Italian Restaurant  0.12
1  Mexican Restaurant  0.08
2         Pizza Place  0.08
3  Dim Sum Restaurant  0.04
4  Salon / Barbershop  0.04

----New Dorp Beach----
                venue  freq
0       Deli / Bodega  0.17
1                Food  0.17
2  

               venue  freq
0        Pizza Place  0.05
1           Pharmacy  0.05
2  Mobile Phone Shop  0.05
3   Greek Restaurant  0.05
4             Bakery  0.05

----Riverdale----
            venue  freq
0            Park  0.22
1     Bus Station  0.11
2  Baseball Field  0.11
3    Home Service  0.11
4            Bank  0.11

----Rochdale----
                             venue  freq
0  Southern / Soul Food Restaurant  0.12
1                           Market  0.12
2                         Bus Stop  0.06
3                   Sandwich Place  0.06
4                         Pharmacy  0.06

----Rockaway Beach----
              venue  freq
0             Beach  0.16
1    Ice Cream Shop  0.06
2  Arepa Restaurant  0.04
3         BBQ Joint  0.04
4       Bus Station  0.04

----Rockaway Park----
                venue  freq
0               Beach  0.16
1          Donut Shop  0.08
2         Pizza Place  0.08
3                Bank  0.08
4  Seafood Restaurant  0.04

----Roosevelt Island----
              

                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.05
2    Sushi Restaurant  0.05
3          Steakhouse  0.05
4            Wine Bar  0.04

----Unionport----
                       venue  freq
0                 Donut Shop  0.09
1  Latin American Restaurant  0.09
2             Ice Cream Shop  0.09
3               Dance Studio  0.04
4          Mobile Phone Shop  0.04

----University Heights----
            venue  freq
0     Pizza Place  0.09
1  Sandwich Place  0.04
2     Bus Station  0.04
3   Burrito Place  0.04
4    Burger Joint  0.04

----Upper East Side----
                  venue  freq
0    Italian Restaurant  0.07
1               Exhibit  0.07
2                Bakery  0.05
3           Coffee Shop  0.05
4  Gym / Fitness Center  0.04

----Upper West Side----
                venue  freq
0  Italian Restaurant  0.06
1         Coffee Shop  0.04
2            Wine Bar  0.04
3                 Bar  0.04
4   Indian Restaurant  0.03

----Utopia----
              ven

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0       Allerton           Pizza Place    Chinese Restaurant   
1       Annadale           Pizza Place              Pharmacy   
2  Arden Heights              Pharmacy         Deli / Bodega   
3      Arlington              Bus Stop   American Restaurant   
4       Arrochar              Bus Stop    Italian Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0         Deli / Bodega           Supermarket        Cosmetics Shop   
1                 Diner          Dance Studio   American Restaurant   
2              Bus Stop           Coffee Shop          Home Service   
3          Intersection         Deli / Bodega           Yoga Studio   
4         Deli / Bodega           Pizza Place          Liquor Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0          Intersection  Fast Food Restaurant     Martial Arts Dojo   
1         Train Station            Restaurant         Deli / Bodega   
2           Pizza Place           Yoga Studio   Filipino Restaurant   
3     Fish & Chips Shop           Event Space               Exhibit   
4            Bagel Shop            Food Truck                 Hotel   

  9th Most Common Venue 10th Most Common Venue  
0         Grocery Store  Check Cashing Service  
1            Sports Bar       Sushi Restaurant  
2           Event Space                Exhibit  
3               Factory     Falafel Restaurant  
4        Sandwich Place  Outdoors & Recreation

### kmeans clustering for neighborhood clustering

In [47]:
# set number of clusters
kclusters = 5

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=7).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 4, 1, 1, 1, 1, 1])

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhoods_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [66]:
neighborhoods_merged.shape

(306, 15)

In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged2['Latitude'], neighborhoods_merged2['Longitude'], neighborhoods_merged2['Neighborhood'], neighborhoods_merged2['Cluster Labels'].astype("int", errors="ignore")):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [73]:
neighborhoods_merged2 = neighborhoods_merged.dropna()
neighborhoods_merged2.shape

(305, 15)